In [28]:
from scipy.stats import chi2_contingency
import calendar
import prince
months = list(map(lambda x: x.lower(), calendar.month_name))[1:]
import pandas as pd
from pandas import DataFrame

import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

import matplotlib.pylab as pylab




params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 10),
          'axes.labelsize': 'x-large',
          'axes.titlesize': 'x-large',
          'xtick.labelsize': 'x-large',
          'ytick.labelsize': 'x-large'}
pylab.rcParams.update(params)

import os

project_folder = f'{os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))}/data/process'
import warnings

warnings.filterwarnings(action='ignore')

from IPython.core.display import display, HTML
# Set the display width to fit the entire notebook width
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.display import Markdown
from IPython.display import display_html
import scipy.stats as stats

predictors ={'Q13':'value_approx','Q14':'is_electric','Q15':'bicycle_type','Q18':'is_recover',  'Q28':'seasons', 'Q29':'purpose', 'age_groups': 'age_groups', 'Q35': 'gender', 'Q36': 'income','Q37':'nm_bikes', 'Q38': 'education','country':'country'}
dependent_vars = {'Q25':'is_replaced','Q30':'mode_alt', 'Q31':'post_act'}
all_vars = predictors.copy()
all_vars.update(dependent_vars)

<span style="color: red;font-size: 50px">RUN THE NEXT CELL FOR THE FIRST TIME ONLY</span>

In [2]:

# Change col names and leave only relevant cols as well as delete unnecessary spaces and parenthesis (except age group)
cols_names = list(all_vars.values())
data_init = pd.read_csv(f'{project_folder}/new_data/new_data_2.csv')
temp = data_init['age_groups']
d_analysis = data_init.rename(columns=all_vars)[cols_names].astype(str).replace(r"\(.*?\)", "").astype(str).replace(":", "").apply(lambda row: [d.split('(')[0].strip() if '(' in d else d.strip() for d in row])
d_analysis['age_groups'] = temp




# This method translate the alternative data into how much the alternative is sustainable
def sus_alter(row):
    alt_stat = ['sustainable','semi','non sustainable']
    if row in  ['Walk','Cycle, personal bicycle','Cycle, rental bicycle','Cycle, public bike share','Micro mobility']:
        return alt_stat[0]
    elif row in ['Transit','Motorcycle or scooter']:
        return alt_stat[1]
    elif row in ['Car, as a driver','Car, as a passenger','Taxi / Ride-hailing service']:
        return alt_stat[2]
    else:
        return row

d_analysis['mode_alt'] = d_analysis['mode_alt'].apply(sus_alter)

d_analysis.to_csv(f'{project_folder}/new_data/new_data_3.csv')



In [8]:
# read data
merge_q = pd.read_csv(f'{project_folder}/new_data/new_data_3.csv')

In [9]:
# organize and populate @more_data  dictionary
more_data ={item:[list(DataFrame(merge_q[item].unique()).dropna()[0]),False] for  item in all_vars.values()}

# Code relevant for reindex
# Bicycle attributes
q= 'value_approx'
df_t = DataFrame(merge_q[q].unique()).dropna().sort_values(by=0, ascending=0).reset_index(drop=True)
more_data[q][0] = pd.concat([df_t.iloc[0], df_t.iloc[6], df_t.iloc[4], df_t.iloc[8], df_t.iloc[7], df_t.iloc[5], df_t.iloc[1:4].sort_values(by=0, ascending=1)])[0].to_list()

# Demographic
more_data['education'][0]= ['Some high school or less',
                            'Graduated high school',
                            'Some university',
                            'Associate’s/vocational/technical\u202fdegree',
                            'Bachelor’s degree',
                            'Graduate degree',
                            'I prefer to not answer',
 ]
q = 'age_groups'
more_data[q][0] = DataFrame(merge_q[q].unique()).dropna()[::-1][0].to_list()


more_data['seasons'][0] = range(3)
q= 'income'
df_t = DataFrame(merge_q[q].unique()).dropna().sort_values(by=0, ascending=0).reset_index(drop=True)
more_data[q][0]= pd.concat([df_t.iloc[0], df_t.iloc[6:2:-1], df_t.iloc[8:6:-1], df_t.iloc[9], df_t.iloc[1:3]])[0].to_list()
q = 'nm_bikes'
df_t = DataFrame(merge_q[q].unique()).dropna().sort_values(by=0, ascending=0).reset_index(drop=True)
more_data[q][0]= pd.concat([df_t.iloc[0], df_t.iloc[3], df_t.iloc[1:3], df_t.iloc[5:], df_t.iloc[4]])[0].to_list()

# for the dependent variable post_act:
more_data['post_act'][0]= ['I stopped cycling',
                            'Less often',
                            'About the same / no change',
                            'More often',
]
# In case where only several cols are relevant
more_data['bicycle_type'][0]= more_data['bicycle_type'][1]=['Hybrid/City/Dutch','Mountain','Road','Gravel/cyclocross']


In [13]:
for name in ['value_approx','income','nm_bikes','age_groups','education']:
    print(more_data[name][0].index('$250-$499'))
    break

1


<span style="color: blue;font-size: 50px">Find relationships between independent variables </span>

In [33]:
dependents_list = ['value_approx','income','nm_bikes','age_groups','education']
data_to_exp = merge_q[dependents_list].fillna(-1)

def to_ordinal(col):
    # This function gets a column and return for each value its ordinal values as it stored in @more_data (for irrelevant data return -1)
    return data_to_exp[col.name].apply(lambda x:more_data[col.name][0].index(x) if x not in ['I prefer to not answer','Don’t know/not sure',-1] else -1)
data_as_ordinal = data_to_exp.apply(to_ordinal)


In [49]:

data_list = []
# The code test every pair of independent variables to find the direction of the correlation
for items in [(x, y) for i,x in enumerate(dependents_list) for y in dependents_list[i+1:]]:
    var_1, var_2 = items
    df = data_as_ordinal[[var_1, var_2 ]]
    couple_to_test = df[~(df== -1).any(axis=1)] # remove missing or irrelevant data
    correlation, p_value = stats.spearmanr(couple_to_test[var_1],couple_to_test[var_2]) # use spearman to test the data
    data_list.append([var_1,var_2,correlation, p_value])
sta_spearman_table = DataFrame(data_list,columns=['var_1','var_2','correlation', 'p_value']).sort_values('correlation',ascending=False)
sta_spearman_table

,var_1,var_2,correlation,p_value
1,value_approx,nm_bikes,0.299765,1.175538e-36
6,income,education,0.281990,2.410065e-29
4,income,nm_bikes,0.241739,8.763972e-22
7,nm_bikes,age_groups,0.233628,1.395895e-22
0,value_approx,income,0.218758,4.698700e-18
5,income,age_groups,0.212985,3.320104e-17
2,value_approx,age_groups,0.191486,1.932273e-16
9,age_groups,education,0.145009,2.140645e-09
8,nm_bikes,education,0.114911,2.286376e-06
3,value_approx,education,0.041716,8.701408e-02


<span style="color: blue;font-size: 50px">####</span>

In [8]:

class MyData:
    """
   The class organizes and explores the data, allowing for the creation of cross-tabulations.
    """
    def __init__(self, var_0, data, com_data):
        """

        :param var_0: The main variable that should not be replaced frequently throughout the program.
        :param data: Data frame
        :param com_data: provides more information when necessary to create more adaptable analysis
        """
        self.cols_name = [var_0,'']
        self.merge_q= data
        self.more_data = com_data
        self.reindex_rows = self.more_data[var_0][0]
    def explore_data(self,cross_tab= True):
        r"""
        Clean the data and print cross_tab if it is required
        :param cross_tab:
        :return:
        """
        # Based on these variables, the analysis can be employed.
        cross_cols_nm =self.cols_name
        cls_to_use = self.more_data[cross_cols_nm[1]][1]
        reindex_temp = self.more_data[cross_cols_nm[1]][0]
        only_rel_f= self.merge_q[cross_cols_nm].dropna()
        if cross_tab:
            if cls_to_use:
                only_rel_f = only_rel_f[only_rel_f[cross_cols_nm[1]].isin(cls_to_use)]
            # For ordinal categories, reindexing the columns is essential to align them in the required order.
            return only_rel_f[cross_cols_nm[1]].value_counts().reindex(reindex_temp), (pd.crosstab(only_rel_f[cross_cols_nm[0]], only_rel_f[cross_cols_nm[1]], normalize='columns') * 100).astype(int).reindex(columns=reindex_temp,index= self.reindex_rows)
        return only_rel_f

    def change_properties(self,name):
        """
        change the dependent variable and update desired reindex list
        :param name:
        :return:
        """
        self.cols_name[0] = name
        self.reindex_rows = self.more_data[name][0]
        sumy =my_data.merge_q[name].value_counts().sum()
        print((my_data.merge_q[name].value_counts()/sumy*100).apply('{:.0f}%'.format))

In [5]:
merge_q.columns

Index(['Unnamed: 0', 'value_approx', 'is_electric', 'bicycle_type',
       'is_recover', 'seasons', 'purpose', 'age_groups', 'gender', 'income',
       'nm_bikes', 'education', 'country', 'is_replaced', 'mode_alt',
       'post_act'],
      dtype='object')

In [9]:

dependent_names = list(dependent_vars.values())+['value_approx','income','nm_bikes','age_groups','education']
predictors_names = list(predictors.values())
my_data = MyData(dependent_names[0],merge_q,more_data)
for dep in dependent_names:
    print(f'\n{dep}\n')
    writer = pd.ExcelWriter(f'output_2/data_with_insight/{dep}.xlsx')
    my_data.change_properties(dep)
    for pre in predictors_names+dependent_names:
        # relevant when we analysis the relationship between dependent_vars
        if pre==dep:
            continue
        my_data.cols_name[1] = pre
        value_counts,cross_tab = my_data.explore_data()
        value_counts.to_excel(writer, sheet_name=pre)
        cross_tab.to_excel(writer, sheet_name=pre, startrow=value_counts.shape[0] + 2)
    writer.save()



is_replaced

​Yes, and I replaced it with exactly what was stolen or something more expensive    46%
No                                                                                  31%
Yes, but the replacement was something cheaper                                      24%
Name: is_replaced, dtype: object

mode_alt

non sustainable            39%
sustainable                36%
Didn’t make those trips    13%
semi                       10%
Don’t know/not sure         2%
Name: mode_alt, dtype: object

post_act

About the same / no change    49%
Less often                    30%
I stopped cycling             15%
More often                     6%
Name: post_act, dtype: object

value_approx

$500-$999                 29%
$1000-$1999               24%
$2000-3999                16%
$250-$499                 13%
$4000-6999                 8%
$7000 or more              5%
Less than $250             4%
Don’t know/not sure        0%
I prefer to not answer     0%
Name: value_approx, dtype: obje

In [ ]:
# ver_to_test =[{'value_approx','income'},{'value_approx','nm_bikes'},{'value_approx','education'},
#               {'nm_bikes','value_approx'},{'nm_bikes','income'},{'nm_bikes','education'},
#               {'income','value_approx'},{'income','nm_bikes'},{'income','nm_bikes'},{'income','age_groups'},{'income','education'},
#               {'age_groups','nm_bikes'},{'age_groups','education'},
#               {'education','income'}]
# cols_to_exclude = ['I prefer to not answer','Don’t know/not sure','Children (<13)']

In [37]:
#
def calculate_chi_cross_tab(two_var_tables):
    r"""
    This function creates new groups based on the data in @vars, cross tab them and calculate Chi-square
    :return:
    """

    # the new names for the variables which be presented on the cross tabulation tabel
    new_names = [t['var'] for t in my_vars]
    two_var_tables[new_names] = two_var_tables.apply(lambda x: [x[t['q']] if x[t['q']] in t['group'] else t['group2'] for t in my_vars],axis=1,result_type='expand')
    cont_table = pd.crosstab([two_var_tables[t['var']] for t in my_vars[:-1]],two_var_tables[my_vars[-1]['var']],normalize='columns')*100

    # perform chi-square test of independence
    chi2, p, dof, expected = chi2_contingency(cont_table)

    # print the test results
    print(f"Chi-square statistic: {chi2:.2f}")
    print(f"P-value: {p:.5f}")
    print(cont_table)
    if to_print:
        cont_table.to_csv(to_print)

def prep_for_chi(only_rel_f,grp:dict):
    r"""
    this function store and set the data so only the relevant data will be used to calculate chi -square
    :param only_rel_f:
    :return:
    """
    # working only with information stored in list in @rel_data for a specific col
    for i in range(1,len(rel_data),2):
        if rel_data[i]:
            only_rel_f= only_rel_f[only_rel_f[rel_data[i-1][0]].isin(rel_data[i])] if rel_data[i-1][1] else only_rel_f[~only_rel_f[rel_data[i-1][0]].isin(rel_data[i])]
    # group data
    for q_temp in grp.keys():
        grp_inf = grp[q_temp]
        only_rel_f[q_temp] = only_rel_f[q_temp].apply(lambda x:grp_inf[0] if x in grp_inf[1] else grp_inf[2])

    calculate_chi_cross_tab(only_rel_f)